# Forrest TDA mapper


In [6]:
import kmapper as km
from kmapper.plotlyviz import *
import sklearn
#from sklearn import ensemble
# DBSCAN from sklearn for clustering algorithms
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

# PCA from sklearn for projection/lens creation
from sklearn.decomposition import PCA
# from sklearn.manifold import MDS
# scipy for interpolation
# import scipy 
# from scipy.interpolate import *
# Plotly and Dash
import plotly.graph_objs as go
from kmapper.plotlyviz import plotlyviz
from kmapper.plotlyviz import *
from ipywidgets import (HBox, VBox)
import dash_html_components as html
import dash_core_components as dcc
import dash
from ipywidgets import interactive, HBox, VBox, widgets, interact
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
print("Imports")

Imports


# Preparation
Time periods creation & obtaining the datasets for each time period and strata and season

In [2]:
predicted_df = pd.read_csv(r"C:\Users\forre\Desktop\REU\TDA\github\UMR-TDA-2021\LTRM data\RF interpolation\water_full.csv")
predicted_df = predicted_df[predicted_df['FLDNUM'] == 'Brighton, IL'] # filter for pool Bellevue, IA is pool 13
### Creating three main time spans and two overlapping time spans,
### a total of five time spans

# defining different time periods
# first decade
time_dec1 = [1993, 1994, 1995, 1997, 1998, 1999, 2000]
# second decade
time_dec2 = [2001, 2002,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]
# third decade
time_dec3 = [2014, 2015, 2016, 2017, 2018, 2019, 2020]
# overlap time periods for continuity
time_overlap1 = [1998, 1999, 2000, 2001, 2002,2003, 2004]
time_overlap2 = [2010, 2011, 2012, 2013, 2014, 2015, 2016]
time_list = [time_dec1, time_overlap1, time_dec2, time_overlap2, time_dec3]

time_list_names = ['93-00', '98-04', '01-13', '10-16', '14-20']        
time_list =[time_dec1, time_overlap1, time_dec2, time_overlap2, time_dec3]
stratum_list = [
    "Main channel","Side channel","Backwater area contiguous to the main channel","Lake Pepin or Swan Lake",
"Impounded",
"Isolated"]
Season_names = [2] # can add other seasons later

nonempty_stratum = []
for i in stratum_list:
    if predicted_df[predicted_df['STRATUM'] == i].shape[0]!=0:
        nonempty_stratum.append(i)
print("Non empty strata are: ",nonempty_stratum) 


df_stratum_season_time_dict_list = []
df_stratum_season_time_dict = {}
s=""

for i in range(len(time_list)): # for every time period
    for j in nonempty_stratum: # for every strata
        for k in Season_names: # for every season # get rid of this line
            s="Stratum " + j + " " + str(k) + " " + time_list_names[i] + str(": ")
            df_stratum_season_time_dict[s] = predicted_df[(predicted_df['YEAR'].isin(time_list[i])) &
                                                             (predicted_df['STRATUM'].isin([j])) & 
                                                             (predicted_df['SEASON'].isin([k]))]
            df_stratum_season_time_dict_list.append(df_stratum_season_time_dict)
            df_stratum_season_time_dict = {}
            # obtain the subdata frame for the year set, strata, and season. Places that in a list
            s=""
            

            
            

print(len(df_stratum_season_time_dict_list))




Non empty strata are:  ['Main channel', 'Side channel', 'Backwater area contiguous to the main channel', 'Lake Pepin or Swan Lake', 'Impounded']
25


In [3]:
predicted_df.shape

(10879, 20)

# TDA mapper function and helpers

In [7]:
def cluster_fun(dict_df, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = 10, PERC_OVERLAP = .3):
    """
    How to cluster the data
    """
    keys = list(dict_df.keys())
    print(keys)
    X = dict_df.get(keys[0])
    X = X[["PREDICTED_WDP", "PREDICTED_SECCHI", "PREDICTED_TEMP", "PREDICTED_DO", 
           "PREDICTED_TURB", "PREDICTED_VEL", "PREDICTED_TP", #"PREDICTED_COND",
           "PREDICTED_TN", "PREDICTED_SS", "PREDICTED_CHLcal"]]
    
    
    if X.shape[0]<DBSCAN_MIN_SAMPLES:
        #print(X)
        print("Not enough data to cluster in ", keys, "_size = ", X.shape[0])
        print("DBSCAN_MIN_SAMPLES", DBSCAN_MIN_SAMPLES)
        return([DBSCAN_MIN_SAMPLES, X.shape[0]])
    
    
    db = DBSCAN(eps=20, min_samples=2).fit(X)
    # Number of clusters in labels, ignoring noise if present.
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    
    return(db)

    
    
def mapper_func(dict_df, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = 10, PERC_OVERLAP =.35, remove_duplicate_nodes = True):
    
    
    #Initialize the dataframe to use 
    keys = list(dict_df.keys())
    print(keys)
    X = dict_df.get(keys[0])
    variables = ["PREDICTED_WDP", "PREDICTED_SECCHI", "PREDICTED_TEMP", "PREDICTED_DO", 
           "PREDICTED_TURB", "PREDICTED_VEL", "PREDICTED_TP", #"PREDICTED_COND",
           "PREDICTED_TN", "PREDICTED_SS", "PREDICTED_CHLcal"]
    
    variables = ["TN", "TP","TEMP", "DO","TURB", "VEL", "SS", "WDP", "CHLcal","SECCHI"] # "COND"
    
    var_to_index = {variables[i]:i for i in range(len(variables))}
    
    projected_vars = variables
    projected_vars_indices = [var_to_index[var] for var in projected_vars]
    
    X = X[variables]
    
    if X.shape[0]<10:
        #print(X)
        print("Not enough data in ", keys, "_size = ", X.shape[0])
        return(X.shape[0])
    # makes indices in kepler mapper 1 to 1 for statistical analysis on nodes in scomplexAnalysis
    
    X.reset_index(drop = True, inplace = True)
    
    # create instance of clustering alg
    cluster_alg = sklearn.cluster.DBSCAN(eps=DBSCAN_EPSILON, min_samples=DBSCAN_MIN_SAMPLES, metric='euclidean')
    # Instantiate kepler mapper object
    mapper = km.KeplerMapper(verbose=0)
    
        
    # defining filter function as projection on to the first component axis
    pca = PCA(n_components=1) # set to 1 for now 
    lens = pca.fit_transform(X)
    print("pca explained variance: ", pca.explained_variance_ratio_)
    
    principle_component = max(abs(pca.components_[0].min()), abs(pca.components_[0].max()))
    max_index = 0
    for i in range(len(pca.components_[0])):
        if abs(pca.components_[0][i]) == principle_component:
            max_index = i
    print("Max principal component: ", variables[max_index])
    
    # summary variables
    summary_variables = mapper.project(np.array(X), projection = projected_vars_indices, scaler = None)
    
    
    # Generate the simplicial complex
    
    
    scomplex = mapper.map(lens, X, cover=km.Cover(n_cubes=N_CUBES, perc_overlap=PERC_OVERLAP), 
                          clusterer=cluster_alg, remove_duplicate_nodes = True)  
    
    
    pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

    color_values = lens [:,0] - lens[:,0].min()
    color_function_name = ['Distance to x-min']
    my_colorscale = pl_brewer
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                     color_values = color_values,  
                                                                     color_function_name=color_function_name, 
                                                                     colorscale=my_colorscale)

    bgcolor = 'rgba(10,10,10, 0.9)'
    # y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

    plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                     factor_size=2.5, edge_linewidth=0.5)
    plot_title = str(keys[0]) + str(DBSCAN_EPSILON) + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    # plot_title = 'Pool 13, Summer 1993-1999; Epsilon ' + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    layout = plot_layout(title=plot_title,  
                         width=620, height=570,
                         annotation_text=get_kmgraph_meta(mapper_summary),  
                         bgcolor=bgcolor)

    # FigureWidget is responsible for event listeners

    fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
    fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
    fw_summary = summary_fig(mapper_summary, height=300)

    dashboard = hovering_widgets(kmgraph, 
                                 fw_graph, 
                                 bgcolor=bgcolor, 
                                 member_textbox_width=600)

    # DESIRED FILE PATH, CHANGE TO FIT YOUR LOCAL MACHINE
    
    directory_path = "Users\\forre\\Desktop\\REU\\TDA\\Data\\TDAOutputs"
    html_output_path = directory_path + "\\" + str(keys[0]) + 'PCA_1' + 'all_var_' + 'Eps_' + str(DBSCAN_EPSILON) +'MinS_' + str(DBSCAN_MIN_SAMPLES) + 'NCUBES_' + str(N_CUBES) + 'PEROvLp_' + str(PERC_OVERLAP) + '.html'
    html_output_path = html_output_path.replace(" ","-")
    html_output_path = html_output_path.replace(":","_")
    html_output_path = "C:\\"+html_output_path
    #print(html_output_path)
    mapper.visualize(scomplex, path_html=html_output_path, color_values = color_values,
                    color_function_name = color_function_name,
                    lens = summary_variables, lens_names = projected_vars)
    
    #Update the fw_graph colorbar, setting its title:
    fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'
    return scomplex, X
    
def mapper_test_func(dict_df, DBSCAN_EPSILON = 10, DBSCAN_MIN_SAMPLES = 5, N_CUBES = [100,100], PERC_OVERLAP = [.20,.20], remove_duplicate_nodes = True):
    
    
    #Initialize the dataframe to use 
#     keys = list(dict_df.keys())
#     print(keys)
#     X = dict_df.get(keys[0])
    X = dict_df
    variables = ["PREDICTED_WDP", "PREDICTED_SECCHI", "PREDICTED_TEMP", "PREDICTED_DO", 
           "PREDICTED_TURB", "PREDICTED_VEL", "PREDICTED_TP", #"PREDICTED_COND",
           "PREDICTED_TN", "PREDICTED_SS", "PREDICTED_CHLcal"]
    
    variables = ["TN", "TP","TEMP", "DO","TURB", "VEL", "SS", "WDP", "CHLcal","SECCHI"] # "COND"
    
    var_to_index = {variables[i]:i for i in range(len(variables))}
    
    projected_vars = variables
    projected_vars_indices = [var_to_index[var] for var in projected_vars]
    
    X = X[variables]
    
    
    Y = X.copy()
    Y = pd.DataFrame(StandardScaler().fit_transform(Y))
    
    if Y.shape[0]<10:
        #print(X)
        print("Not enough data in ", keys, "_size = ", X.shape[0])
        return(Y.shape[0])
    # makes indices in kepler mapper 1 to 1 for statistical analysis on nodes in scomplexAnalysis
    X.reset_index(drop = True, inplace = True)

    Y.reset_index(drop = True, inplace = True)
    
    # create instance of clustering alg
    cluster_alg = sklearn.cluster.DBSCAN(eps=DBSCAN_EPSILON, min_samples=DBSCAN_MIN_SAMPLES, metric='euclidean')
    # Instantiate kepler mapper object
    mapper = km.KeplerMapper(verbose=0)
    
        
    # defining filter function as projection on to the first component axis
    pca = PCA(n_components=2) # set to 1 for now 
    lens = pca.fit_transform(X)
    print("pca explained variance: ", pca.explained_variance_ratio_)
    
    principle_component = max(abs(pca.components_[0].min()), abs(pca.components_[0].max()))
    max_index = 0
    for i in range(len(pca.components_[0])):
        if abs(pca.components_[0][i]) == principle_component:
            max_index = i
    print("Max principal component: ", variables[max_index])
    
    # summary variables
    summary_variables = mapper.project(np.array(X), projection = projected_vars_indices, scaler = None)
    
    
    # Generate the simplicial complex
    # ******** # This is the Robust Scaler Change
    
    scomplex = mapper.map(lens, Y, cover=km.Cover(n_cubes=N_CUBES, perc_overlap=PERC_OVERLAP), 
                          clusterer=cluster_alg, remove_duplicate_nodes = True)  
    
    
    pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

    color_values = lens [:,0] - lens[:,0].min()
    color_function_name = ['Distance to x-min']
    my_colorscale = pl_brewer
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                     color_values = color_values,  
                                                                     color_function_name=color_function_name, 
                                                                     colorscale=my_colorscale)

    bgcolor = 'rgba(10,10,10, 0.9)'
    # y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

    plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                     factor_size=2.5, edge_linewidth=0.5)
    plot_title = str(DBSCAN_EPSILON) + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    # plot_title = 'Pool 13, Summer 1993-1999; Epsilon ' + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    layout = plot_layout(title=plot_title,  
                         width=620, height=570,
                         annotation_text=get_kmgraph_meta(mapper_summary),  
                         bgcolor=bgcolor)

    # FigureWidget is responsible for event listeners

    fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
    fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
    fw_summary = summary_fig(mapper_summary, height=300)

    dashboard = hovering_widgets(kmgraph, 
                                 fw_graph, 
                                 bgcolor=bgcolor, 
                                 member_textbox_width=600)

    # DESIRED FILE PATH, CHANGE TO FIT YOUR LOCAL MACHINE
    
    directory_path = "Users\\forre\\Desktop\\REU\\TDA\\Data\\TDAOutputs\\LetsGoAgain"
    html_output_path = directory_path + "\\"+ 'PCA_2' + 'all_var_' + 'Eps_' + str(DBSCAN_EPSILON) +'MinS_' + str(DBSCAN_MIN_SAMPLES) + 'NCUBES_' + str(N_CUBES) + 'PEROvLp_' + str(PERC_OVERLAP) + '.html'
    html_output_path = html_output_path.replace(" ","")
    html_output_path = html_output_path.replace(":","_")
    html_output_path = "C:\\"+html_output_path
    #print(html_output_path)
    mapper.visualize(scomplex, path_html=html_output_path, color_values = color_values,
                    color_function_name = color_function_name,
                    lens = summary_variables, lens_names = projected_vars)
    
    #Update the fw_graph colorbar, setting its title:
    fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'
    return scomplex, X, Y
    


    
    
    
    
print("Functions loaded")

Functions loaded


In [8]:
scomplex, X, Y = mapper_test_func(predicted_df)


pca explained variance:  [0.7692028  0.15181453]
Max principal component:  SS


In [9]:
scomplex, X, Y = mapper_test_func(predicted_df)
import json as js
json = js.dumps(scomplex)
f = open(r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\LetsGoAgain\pool26_complex.json","w")
f.write(json)
f.close()
import pickle as pk
pickle = pk.dump(X, open(r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\LetsGoAgain\pool26_df.p","wb"))
pickle2 = pk.dump(Y, open(r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\LetsGoAgain\pool26_df_StandardScaled.p","wb"))

pca explained variance:  [0.7692028  0.15181453]
Max principal component:  SS


In [ ]:
mapper_output_dict = {}
mapper_output_df = {}
for i in df_stratum_season_time_dict_list:
            scomplex, X = mapper_func(i)
            mapper_output_dict[str(list(i.keys()))] = scomplex
            mapper_output_df[str(list(i.keys()))] = X

In [ ]:
import json as js
json = js.dumps(mapper_output_dict)
f = open(r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\PCA1_10Cubes_30Perc_complices.json","w")
f.write(json)
f.close()

In [ ]:
mapper_output

In [ ]:
import pickle as pk
pickle = pk.dump(mapper_output_df,
                 open(r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\PCA1_10Cubes_30Perc_complices_dfs.p","wb"))